## Read data from train.txt and filter it from unwanted patterns


In [1]:
import numpy as np
from keras.utils import to_categorical
from Embeddings import Word2Vec, FastText
from Preprocessing import utils, character_encoding
from Models import rnn,lstm
import config as conf

config = conf.ConfigLoader().load_config()

 
Configurations : 
number_test_of_words : 20000
number_validation_of_words : 3000
classifier : lstm
embedding : fasttext
is_training : True
embedding_vector_size : 200
batch_size : 64
num_epochs : 7


## Preprocessing


Clean data from special characcters


In [2]:
training_set = utils.read_data(f"./Dataset/train.txt")
print("training_set", training_set[0:500])
print('\n')
filtered_training_set = utils.filter_data(training_set)
print("filtered_training_set", filtered_training_set[0:500])


training_set قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ( 14 / 123 )
ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً ( كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ) ابْنُ عَرَفَةَ : قَوْلُ ابْنِ شَاسٍ : أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ ( وَسِحْرٍ ) مُحَمَّدٌ : قَوْلُ مَالِكٍ و


filtered_training_set قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِك

In [3]:
validation_set = utils.read_data(f"./Dataset/val.txt")
print("validation_set", validation_set[0:500])
print('\n')
filtered_validation_set = utils.filter_data(validation_set)
print("filtered_validation_set", filtered_validation_set[0:500])


validation_set ( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .
( الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ قَاعِدَةِ تَقَدُّمِ الْحُكْمِ عَلَى سَبَبِهِ دُونَ شَرْطِهِ أَوْ شَرْطِهِ دُونَ سَبَبِهِ وَبَيْنَ قَاعِدَةِ تَقَدُّمِهِ عَلَى السَّبَبِ وَالشَّرْطِ جَمِيعًا ) وَتَحْرِيرُهُ أَنَّ الْحُكْمَ إنْ كَانَ لَهُ سَبَبٌ بِغَيْرِ شَرْطٍ فَتَقَدَّمَ عَلَيْهِ لَا يُعْتَبَرُ أَوْ كَانَ لَهُ سَبَبَانِ أَوْ أَسْبَابٌ فَتَقَدَّمَ عَلَى جَمِيعِهَا لَمْ يُعْتَبَرْ أَوْ عَلَى بَعْضِهَا دُونَ بَعْضٍ اُعْتُبِرَ بِنَاءً عَلَى


filtered_validation_set  قَوْلُهُ وَلَا تُكْرَهُ ضِيَافَتُهُ 
 الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ قَاعِدَةِ تَقَدُّمِ الْحُكْمِ عَلَى سَبَبِهِ دُونَ شَرْطِهِ أَوْ شَرْطِهِ دُونَ سَبَبِهِ وَبَيْنَ قَاعِدَةِ تَقَدُّمِهِ عَلَى السَّبَبِ وَالشَّرْطِ جَمِيعًا وَتَحْرِيرُهُ أَنَّ الْحُكْمَ إنْ كَانَ لَهُ سَبَبٌ بِغَيْرِ شَرْطٍ فَتَقَدَّمَ عَلَيْهِ لَا يُعْتَبَرُ أَوْ كَانَ لَهُ سَبَبَانِ أَوْ أَسْبَابٌ فَتَقَدَّمَ عَلَى جَمِيعِهَا لَمْ يُعْتَبَرْ أَوْ عَلَى بَعْضِهَا دُونَ ب

Splite Training data and Validation data into words then separate diacritics from each word


In [4]:
# Preparing Training Set
words_set_train = utils.split_data_to_words(filtered_training_set)
num_of_words_train = config['number_test_of_words']
text_without_diacritics, diacritic_list = character_encoding.PrepareData(words_set_train[0:num_of_words_train])


# Assume this is a validation set
words_set_val = utils.split_data_to_words(filtered_validation_set)
num_of_words_val = config['number_validation_of_words']
text_without_diacritics_validation, diacritic_list_validation = character_encoding.PrepareData(words_set_val[0:num_of_words_val])

## Feature Extraction


Split training data to sentences and remove diacritics from each sentence


In [5]:
sentences = utils.split_data_to_sentences(filtered_training_set)
list_of_sentences = character_encoding.RemoveDiacriticFromSentence(sentences)

Choose embedding model from `config.json file`


In [6]:
# Choose embedding model from config file
if config["embedding"] == "word2vec":
    file_path = './Embeddings/word2vec_model.bin'
    embedding_model = Word2Vec.W2V(list_of_sentences, vector_size = config["embedding_vector_size"])

elif config["embedding"] == "fasttext":
    file_path = './Embeddings/fasttext_model.bin'
    embedding_model = FastText.FastTextEmbedding(list_of_sentences, vector_size = config["embedding_vector_size"])

else:
    raise Exception("Invalid embedding type")

Training the word embedding model or load it (if already saved)


In [7]:

is_training = config["is_training"]    # Change this to False if you want to load the model and not train it again

if embedding_model.is_model_saved(file_path) and is_training == False:
    embedding_model.load_model(file_path)
else:
    embedding_model.train()
    embedding_model.save_model(file_path)

FastText model saved to:  ./Embeddings/fasttext_model.bin


Concatinate **Training** Word embeddings + Characted Embedding


In [8]:

concatinated_vector_train, diacritic_list = utils.concatinate_word_char_embeddings(text_without_diacritics, diacritic_list, embedding_model = embedding_model)

# calculate total character diacritic list for the assert
count_train = 0
for d in diacritic_list:
    count_train += len(d)  
assert (len(concatinated_vector_train) == count_train), f"Error : Train Set Len ({len(concatinated_vector_train)}) != Len diacritic ({count_train}) list have different sizes, "


Concatinate **Validation** Word embeddings + Characted Embedding


In [9]:
concatinated_vector_validation, diacritic_list_validation = utils.concatinate_word_char_embeddings(text_without_diacritics_validation, diacritic_list_validation, embedding_model = embedding_model)

count_validation = 0
for d in diacritic_list_validation:
    count_validation += len(d) 

assert (len(concatinated_vector_validation) == count_validation), f"Error : validation Set Len ({len(concatinated_vector_validation)}) != Len diacritic ({count_validation}) list have different sizes, "

## Building The Model


In [10]:
input_size = len(concatinated_vector_train[0])
output_size = len(character_encoding.DIACRITICS)
print("input size : ", input_size)
print("output size : ", output_size)

input size :  236
output size :  16


Choose Classification model from `config.json file`


In [11]:
# Create an instance of the LSTM class
if config["classifier"] == "lstm":
    model = lstm.LSTM_Model(input_shape=(None, 1), output_shape = output_size)

elif config["classifier"] == "rnn":
    model = rnn.RNN(input_shape=(None, 1), output_shape = output_size)
    
else:
    raise Exception("Invalid model type")

c:\Users\marky\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Preparing Training data to be passed into the `model.train()`


In [12]:
# Convert the training data to the required format
X_train = concatinated_vector_train # np.array([[[character_encoding.CharToOneHOt(char)]] for word in text_without_diacritics for char in word])

y_train = []
for word_diacritic in diacritic_list:
    for diacritic in word_diacritic:
        #print(utils.map_text_to_diacritic(diacritic))
        index = character_encoding.DIACRITICS.index(diacritic)
        y_train.append(to_categorical(index, num_classes=output_size))
y_train = np.array(y_train)
X_train = np.array(X_train)
print("X_train size : ", X_train.shape)
print("y_train size : ", y_train.shape)

X_train size :  (78956, 236)
y_train size :  (78956, 16)


In [13]:
# Train the model
model.train(X_train, y_train, epochs = config["num_epochs"], batch_size = config["batch_size"])

Epoch 1/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 100s 76ms/step - accuracy: 0.3440 - loss: 1.8620
Epoch 2/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 83s 68ms/step - accuracy: 0.3818 - loss: 1.6758
Epoch 3/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 83s 68ms/step - accuracy: 0.3992 - loss: 1.5976
Epoch 4/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 84s 68ms/step - accuracy: 0.3967 - loss: 1.5677
Epoch 5/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 83s 67ms/step - accuracy: 0.4020 - loss: 1.5433
Epoch 6/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 84s 68ms/step - accuracy: 0.4078 - loss: 1.5310
Epoch 7/7
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 85s 69ms/step - accuracy: 0.4044 - loss: 1.5152


## Model Evaluation


Preparing Validation data to be passed into the `model.evaluate()`


In [14]:
# Convert the validation data to the required format
X_validation = concatinated_vector_validation # np.array([[[character_encoding.CharToOneHOt(char)]] for word in text_without_diacritics_validation for char in word])
y_validation = []
for word_diacritic in diacritic_list_validation:
    for diacritic in word_diacritic:
        index = character_encoding.DIACRITICS.index(diacritic)
        y_validation.append(to_categorical(index, num_classes=output_size))

y_validation = np.array(y_train)
X_validation = np.array(X_train)
print("X_validation size : ", X_validation.shape)
print("y_validation size : ", y_validation.shape)

X_validation size :  (78956, 236)
y_validation size :  (78956, 16)


In [15]:
# Evaluate the model
lost , accuracy = model.evaluate(X_validation, y_validation)

2468/2468 ━━━━━━━━━━━━━━━━━━━━ 53s 21ms/step - accuracy: 0.4110 - loss: 1.5017


# Testing


testing on a given sentence


In [16]:
# Predict the diacritics of the validation data
sentence_test = "يأخذون بعض ما تيسر لهم أخذه فيختلسونه و يجعلونه تحتهم حتى إذا رجعوا إلى بيوتهم أخرجوه"
sentence_words = utils.split_data_to_words(sentence_test)

x_test, _ = utils.concatinate_word_char_embeddings(sentence_words, diacritic_list, embedding_model = embedding_model)
# print("x_test : ", x_test)
x_test = np.array(x_test)
print("x_test len : ", len(sentence_test))
print("X_test size : ", x_test.shape)

x_test len :  85
X_test size :  (70, 236)


In [17]:
y_pred = model.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


In [18]:
# something is wrong here
print("X : ", sentence_test)
print("Len X  : ", len(sentence_test))
print("Len Y predicted  : ", len(y_pred))

# Print the predicted diacritics
diacritics_pred = []
for pred in y_pred:
    index = np.argmax(pred)
    diac = character_encoding.DIACRITICS[index]
    print(character_encoding.diacritic_to_str(diac))
    diacritics_pred.append(character_encoding.diacritic_to_str(diac))

# character_encoding.print_text_to_diacritic_mapping(sentence_test, diacritics_pred, True)

X :  يأخذون بعض ما تيسر لهم أخذه فيختلسونه و يجعلونه تحتهم حتى إذا رجعوا إلى بيوتهم أخرجوه
Len X  :  85
Len Y predicted  :  70
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
SUKUN
SUKUN
SUKUN
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
KASRA
KASRA
KASRA
KASRA
KASRA
SHADDA_FATHA
SHADDA_FATHA
SHADDA_FATHA
 
 
 
FATHA
FATHA
FATHA
FATHA
FATHA
 
 
 
KASRA
KASRA
KASRA
KASRA
KASRA
KASRA
FATHA
FATHA
FATHA
FATHA
FATHA
FATHA
